In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime, date
from psx import stocks, tickers
from IPython.display import display
import pandas as pd
import investpy
import yfinance as yf
from matplotlib.dates import relativedelta
from apps.environment.models import Stock
from apps.environment.models.stock import StockBuffer
from apps.environment.models.commodity import Commodity
from apps.environment.models.commodity import CommodityBuffer
import pytz
from os import environ
from math import ceil, floor
import psycopg2



In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Assuming tickers is your DataFrame


tickers_column = tickers()
# display(tickers_column)
symbols = list(tickers_column["symbol"])
names = list(tickers_column["name"])
len_syms = len(symbols)
# # print(symbols)
hundred_inds = pd.read_csv('KSE100 indx Comps - stock-exchange-kse-100pakistan.csv').to_dict(orient='records')
# print(hundred_inds)
hun_syms = {}

all_syms = {}
for ind in range(len_syms):
    all_syms[symbols[ind]] = names[ind]

the_index = 1
for hun_ind in hundred_inds:
    if hun_ind["SYMBOL"] in all_syms:
        sym_b = hun_ind["SYMBOL"]
        if sym_b not in hun_syms:
            hun_syms[sym_b] = {
                "index": the_index,
                "name": all_syms[sym_b]
            }
            the_index = the_index + 1

db_stcks = {}
all_stcks = Stock.objects.all()
max_indx = len(all_stcks)
for astck in all_stcks:
    db_stcks[astck.symbol] = astck.symbol

new_stcks_list = []
for sym_bl in hun_syms:
    if sym_bl not in db_stcks:
        sym_data = hun_syms[sym_bl]
        new_indx = sym_data["index"]
        if new_indx<=max_indx:
            new_indx = max_indx+1
        max_indx = max_indx+1
        new_stcks_list.append(
            Stock(
                index=sym_data["index"],
                symbol=sym_bl,
                name=sym_data["name"],
                type=100
            )
        )

Stock.objects.bulk_create(new_stcks_list)

In [ ]:


all_stocks = Stock.objects.all()

stock_syms = {}

for a_stock in all_stocks:
    if a_stock.symbol not in stock_syms:
        stock_syms[a_stock.symbol] = {
            "object": a_stock,
            "snapshots": {},
            "last_date": None
        }

snapshots = StockBuffer.objects.select_related("stock").all()
for snapshot in snapshots:
    snp_stck = snapshot.stock
    snp_stck_symbl = snp_stck.symbol
    if snp_stck_symbl in stock_syms:
        snp_data = stock_syms[snp_stck_symbl]
        stck_snps = snp_data["snapshots"]
        if snapshot.captured_at not in stck_snps:
            stck_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not snp_data["last_date"]:
                snp_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for stck_sym, stck_data in stock_syms.items():
    new_snapshots = []
    last_date: datetime = stck_data["last_date"]
    if last_date is None:
        start_from = date(2020, 1, 1)
    else:
        start_from = last_date
    print(last_date, stck_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = stocks(stck_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    stck_snpshts = stck_data["snapshots"]
    the_stock: Stock = stck_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in stck_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(StockBuffer(
                stock = the_stock,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
                change = change,
                volume = data_point.Volume,
                bid_vol = 100000000,
                bid_price = nw_price,
                offer_vol = 100000000,
                offer_price = nw_price,
                open = data_point.Open,
                close = nw_price,
                high = data_point.High,
                low = data_point.Low
            ))
        last_data_point = data_point
    StockBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:


all_commodities = Commodity.objects.all()

commodity_syms = {}

for a_commodity in all_commodities:
    if a_commodity.symbol not in commodity_syms:
        commodity_syms[a_commodity.symbol] = {
            "object": a_commodity,
            "snapshots": {},
            "last_date": None
        }

snapshots = CommodityBuffer.objects.select_related("commodity").all()
for snapshot in snapshots:
    snp_cmmdty = snapshot.commodity
    snp_cmmdty_symbl = snp_cmmdty.symbol
    if snp_cmmdty_symbl in commodity_syms:
        cmmdty_data = commodity_syms[snp_cmmdty_symbl]
        cmmdty_snps = cmmdty_data["snapshots"]
        if snapshot.captured_at not in cmmdty_snps:
            cmmdty_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not cmmdty_data["last_date"]:
                cmmdty_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for cmmdty_sym, cmmdty_data in commodity_syms.items():
    new_snapshots = []
    last_date: datetime = cmmdty_data["last_date"]
    if last_date is None:
        start_from = date(2019, 12, 30)
    else:
        start_from = date(2019, 12, 30)
        # start_from = last_date
    print(last_date, cmmdty_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = yf.download(cmmdty_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    cmmdty_snpshts = cmmdty_data["snapshots"]
    the_commodity: Commodity = cmmdty_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in cmmdty_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(CommodityBuffer(
                commodity = the_commodity,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
            ))
        last_data_point = data_point
    CommodityBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [3]:
starting_time_step = datetime(year=2020, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

In [4]:
the_current_time_step = env_config.get("the_current_time_step")
max_epi_len = 50

__last_time_step = the_current_time_step + relativedelta(days=max_epi_len)
the_current_time_step = pytz.utc.localize(datetime.strptime(str(the_current_time_step), '%Y-%m-%d %H:%M:%S'))
__last_time_step = pytz.utc.localize(datetime.strptime(str(__last_time_step), '%Y-%m-%d %H:%M:%S'))
str_time_step = str(the_current_time_step)

str_last_time_step = str(__last_time_step)
db_params = env_config.get("db_params")
db_conn = psycopg2.connect(**db_params)
cursor = db_conn.cursor()

stcks_query = """
    SELECT *
    FROM stocks
"""
cursor.execute(stcks_query)
stck_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stck_df = pd.DataFrame(stck_data, columns=column_names) 
# stck_df = stck_df.to_dict(orient='records')

stcks_buffer_query = f"""
    SELECT stocks_buffers.*, stocks.index
    FROM stocks_buffers 
    JOIN stocks on stocks.id = stocks_buffers.stock_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(stcks_buffer_query)
stcks_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stcks_buffer_df = pd.DataFrame(stcks_buffer_data, columns=column_names)

# stcks_buffer_df = stcks_buffer_df.to_dict(orient='records')

cmmdties_buffer_query = f"""
    SELECT commodities_buffers.*, commodities.index
    FROM commodities_buffers 
    JOIN commodities on commodities.id = commodities_buffers.commodity_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(cmmdties_buffer_query)
cmmdties_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_buffer_df = pd.DataFrame(cmmdties_buffer_data, columns=column_names)
# cmmdties_buffer_df = cmmdties_buffer_df.to_dict(orient='records')

cmmdties_query = """
    SELECT *
    FROM commodities
"""
cursor.execute(cmmdties_query)
cmmdties_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_df = pd.DataFrame(cmmdties_data, columns=column_names) 

In [23]:
# Ensure that the 'captured_at' column is in datetime format
stcks_buffer_df['captured_at'] = pd.to_datetime(stcks_buffer_df['captured_at'])

# Create a list to store the final DataFrames
result_datasets = []

# Group by stock_id and iterate over each group
for stock_id, group in stcks_buffer_df.groupby('stock_id'):
    # Sort the group by 'captured_at' to ensure it's ordered
    group = group.sort_values(by='captured_at')

    # Fill missing values with the previous day's values
    group_filled = group.fillna(method='ffill')

    # Fill remaining missing values with the next day's values
    group_filled = group_filled.fillna(method='bfill')

    # Append the filled group to the result_datasets list
    result_datasets.append(group_filled)

# Concatenate the datasets into a single DataFrame
final_stocks_buffer_df = pd.concat(result_datasets)


print(len(final_stocks_buffer_df))
display(final_stocks_buffer_df)


/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/2991066917.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group_filled = group.fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/2991066917.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group_filled = group_filled.fillna(method='bfill')


3215


,id,captured_at,price_snapshot,change,volume,bid_vol,bid_price,offer_vol,offer_price,stock_id,close,high,low,open,index
14,49353,2020-01-01 00:00:00+00:00,3.76,0.000000,1263500.0,100000000.0,3.76,100000000.0,3.76,1,3.76,3.90,3.11,3.11,1
15,49354,2020-01-02 00:00:00+00:00,4.02,0.069149,1561000.0,100000000.0,4.02,100000000.0,4.02,1,4.02,4.25,3.70,3.84,1
16,49355,2020-01-03 00:00:00+00:00,4.07,0.012438,666500.0,100000000.0,4.07,100000000.0,4.07,1,4.07,4.20,3.91,4.10,1
17,49356,2020-01-06 00:00:00+00:00,4.00,-0.017199,664500.0,100000000.0,4.00,100000000.0,4.00,1,4.00,4.04,3.83,3.99,1
18,49357,2020-01-07 00:00:00+00:00,4.34,0.085000,861000.0,100000000.0,4.34,100000000.0,4.34,1,4.34,4.40,3.95,4.14,1
19,49358,2020-01-08 00:00:00+00:00,4.03,-0.071429,1014500.0,100000000.0,4.03,100000000.0,4.03,1,4.03,4.48,3.50,4.00,1
20,49359,2020-01-09 00:00:00+00:00,4.63,0.148883,2050500.0,100000000.0,4.63,100000000.0,4.63,1,4.63,4.88,4.30,4.35,1
21,49360,2020-01-10 00:00:00+00:00,4.65,0.004320,591000.0,100000000.0,4.65,100000000.0,4.65,1,4.65,5.00,4.62,4.84,1
22,49361,2020-01-13 00:00:00+00:00,4.41,-0.051613,765500.0,100000000.0,4.41,100000000.0,4.41,1,4.41,4.75,4.30,4.65,1
23,49362,2020-01-14 00:00:00+00:00,4.36,-0.011338,237500.0,100000000.0,4.36,100000000.0,4.36,1,4.36,4.55,4.31,4.45,1
